#0. 환경

In [0]:
# %pip install optuna

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, regexp_extract, regexp_replace, row_number, desc, udf, lit
from pyspark.sql.functions import struct, array_contains, collect_set, collect_list
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, VectorAssembler, MinMaxScaler, StringIndexer
from pyspark.ml import Pipeline
import numpy as np
import pandas as pd
from xgboost.spark import SparkXGBClassifier
import mlflow
import mlflow.sklearn
import optuna

In [0]:
spark = SparkSession.builder \
    .appName("sparkXGBoost") \
    .getOrCreate()
mlflow.autolog(disable=True)
mlflow.spark.autolog(disable=True)

# 1. 데이터로드

### 데이터 로드

In [0]:
catalog = "1dt_team8_databricks"
schema = "`final`"
path = f"{catalog}.{schema}"

try:
    train = spark.read.table(f"{path}.train_df")
    validation = spark.read.table(f"{path}.validation_df")
    test = spark.read.table(f"{path}.test_df")
except Exception as e:
    print(f"Error loading data from Unity Catalog Volume: {e}")
# display(train)
# display(validation)
# display(test)

train = train.withColumn("label", when(train["rating"] >= 4, 1).otherwise(0))
validation = validation.withColumn("label", when(validation["rating"] >= 4, 1).otherwise(0))
test = test.withColumn("label", when(test["rating"] >= 4, 1).otherwise(0))

In [0]:
train.show(3)

### IMDB 로드

In [0]:
catalog = "1dt_team8_databricks"
schema = "`imdb`"
imdb_path = f"{catalog}.{schema}"

try:
    imdb_ratings = spark.read.table(f"{imdb_path}.title_ratings")
    print("Data loaded successfully from Unity Catalog Volume.")
except Exception as e:
    print(f"Error loading data from Unity Catalog Volume: {e}")
    print(f"Please ensure CSV files (imdb_ratings.csv) exist in {imdb_path}")

# 2. 데이터분리

## 파이프라인

In [0]:
tokenizer = RegexTokenizer(inputCol="genres", outputCol="genres_tokens", pattern="\\|")
vectorizer = CountVectorizer(inputCol="genres_tokens", outputCol="genres_vec")
# assembler_numvotes = VectorAssembler(inputCols=["numVotes"], outputCol="numVotes_vec")
# scaler = MinMaxScaler(inputCol="numVotes_vec", outputCol="numVotes_scaled")
user_indexer = StringIndexer(inputCol="userId", outputCol="userIndex")
user_indexer_model = user_indexer.fit(train)
movie_indexer = StringIndexer(inputCol="movieId", outputCol="movieIndex")

assembler_all = VectorAssembler(
    # inputCols=["genres_vec", "averageRating", "numVotes_scaled", "userIndex", "movieIndex", "year"],
    inputCols=["genres_vec", "userIndex", "movieIndex"],
    outputCol="features"
)

pipeline = Pipeline(stages=[
    # tokenizer, vectorizer, assembler_numvotes, scaler,
    # user_indexer, movie_indexer, assembler_all
    tokenizer, vectorizer, 
    user_indexer, movie_indexer, assembler_all
])

pipeline_model = pipeline.fit(train)
train_transformed = pipeline_model.transform(train)
validation_transformed = pipeline_model.transform(validation)
test_transformed = pipeline_model.transform(test)

In [0]:
train_data_with_val = train_transformed.withColumn("validationIndicator", lit(0))
validation_data_with_val = validation_transformed.withColumn("validationIndicator", lit(1))
train_val_union = train_data_with_val.union(validation_data_with_val)

# 3. 모델 설계 및 평가

## 평가지표 정의

In [0]:
# Precision@K 계산 함수
def precision_at_k(df: pd.DataFrame, k: int = 5, threshold: float = 4.0) -> float:
    top_k = (
        df.sort_values(["userIndex", "y_pred_proba"], ascending=[True, False])
        .groupby("userIndex")
        .head(k)
    )
    top_k["hit"] = (top_k["y_true"] >= threshold).astype(int)
    user_precision = top_k.groupby("userIndex")["hit"].mean()
    return user_precision.mean()

In [0]:
def recall_at_k(df: pd.DataFrame, k: int = 5, threshold: float = 4.0) -> float:
    # 사용자별 실제 긍정(평점 >= threshold) 아이템 수
    user_positive_counts = df[df["y_true"] >= threshold].groupby("userIndex").size()

    # 사용자별 top-k 추천 추출
    top_k = (
        df.sort_values(["userIndex", "y_pred_proba"], ascending=[True, False])
        .groupby("userIndex")
        .head(k)
        .copy()
    )
    top_k["hit"] = (top_k["y_true"] >= threshold).astype(int)

    # 사용자별 top-k 추천에서 맞춘 아이템 수
    user_hits = top_k.groupby("userIndex")["hit"].sum()

    # 사용자별 recall 계산 (실제 긍정 아이템 수가 0인 경우 제외)
    recall_per_user = user_hits / user_positive_counts
    recall_per_user = recall_per_user.dropna()  # NaN 처리 (실제 긍정 아이템이 없는 사용자 제외)

    # 전체 사용자 평균 Recall@K 반환
    return recall_per_user.mean()

In [0]:
def ndcg_at_k(df: pd.DataFrame, k: int = 10) -> float:
    def dcg(relevances):
        return np.sum((2**relevances - 1) / np.log2(np.arange(2, relevances.size + 2)))

    ndcgs = []

    for user, group in df.groupby("userIndex"):
        # user별 y_pred_proba 기준 내림차순 정렬, top-k 추출
        top_k = group.sort_values("y_pred_proba", ascending=False).head(k)
        rel = top_k["y_true"].values
        
        # 이상적인 순서(내림차순 y_true 정렬)
        ideal_rel = np.sort(group["y_true"].values)[::-1][:k]
        
        actual_dcg = dcg(rel)
        ideal_dcg = dcg(ideal_rel)

        ndcg = actual_dcg / ideal_dcg if ideal_dcg > 0 else 0
        ndcgs.append(ndcg)

    return np.mean(ndcgs)

## XgbClassifier

### 모델 설계 및 실험

In [0]:
xgb_model = SparkXGBClassifier(
    max_depth=6,
    num_round=100,
    eta=0.1,
    early_stopping_rounds=50,
    eval_metric='logloss',
    missing=0.0,  # NaN 처리 방식
    features_col="features",
    label_col="label",
    prediction_col="prediction",
    probability_col="probability",
    seed=0,
    validation_indicator_col="validationIndicator"
)

# 2. 학습
xgb_model_fitted = xgb_model.fit(train_val_union)

In [0]:
train_val_union.select("validationIndicator").distinct().show()
train_val_union.select("validationIndicator").printSchema()

In [0]:
# 1. 파티션 수 줄이기
train_val_union = train_val_union.repartition(4)

# 2. 단순한 모델 파라미터
xgb_model = SparkXGBClassifier(
    max_depth=6,
    num_round=100,
    eta=0.1,
    features_col="features",
    label_col="label",
    seed=0
)

# 3. 샘플 데이터로 실행 테스트 (예: 1000개)
train_val_sample = train_val_union.limit(1000)

xgb_model_fitted = xgb_model.fit(train_val_sample)

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.types import IntegerType

train_data_with_val = train_transformed.withColumn("validationIndicator", lit(0).cast(IntegerType()))
validation_data_with_val = validation_transformed.withColumn("validationIndicator", lit(1).cast(IntegerType()))

train_val_union = train_data_with_val.union(validation_data_with_val)

# 다시 사용하되, num_workers 명시
xgb_model = SparkXGBClassifier(
    max_depth=6,
    num_round=100,
    eta=0.1,
    features_col="features",
    label_col="label",
    seed=0,
    validation_indicator_col="validationIndicator",
    num_workers=2  # 꼭 명시
)
xgb_model_fitted = xgb_model.fit(train_val_union)

In [0]:
# 3. 예측
predictions = xgb_model_fitted.transform(test_transformed)

# 4. 확률값 추출 (예: Precision@K 등 평가용)
predictions = predictions.withColumn("y_pred_proba", col("probability")[1]) \
                         .withColumnRenamed("label", "y_true")

# 5. Pandas로 변환 (위에서 사용한 평가지표 함수 사용 위해)
test_pdf = predictions.select("userIndex", "movieIndex", "y_true", "prediction", "y_pred_proba").toPandas()

### 평가지표

In [0]:
p_at_10 = precision_at_k(test_pdf, k=10)
print(f"Precision@10: {p_at_10:.4f}")

r_at_10 = recall_at_k(test_pdf, k=10)
print(f"Recall@10: {r_at_10:.4f}")

ndcg_10 = ndcg_at_k(test_pdf, k=10)
print(f"NDCG@10: {ndcg_10:.4f}")

### 사용자별 추천 영화

In [0]:
K = 10

# 1. 사용자별 예측 상위 K개 영화 선택
window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_proba").desc())

ranked = test_pred.withColumn("rank", row_number().over(window_spec)) \
                  .filter(col("rank") <= K)

# 2. 추천 영화 정보 묶기 (movieIndex와 예측 확률)
ranked = ranked.withColumn("recommendation", struct(col("movieIndex").alias("movieId"), col("y_pred_proba").alias("pred_rating")))

# 3. 사용자별로 리스트로 그룹화
recommendations = ranked.groupBy("userIndex") \
    .agg(collect_list("recommendation").alias("recommendations")) \
    .orderBy("userIndex")

display(recommendations)

In [0]:
K = 10

# 1. 영화 메타데이터에서 movieId와 title 추출
# 예: movies라는 테이블이나 DataFrame이 있다고 가정
movies_meta = train.select("movieId", "title").dropDuplicates()

# 2. test_pred에 title 붙이기 (movieId 기준 조인)
test_with_title = test_pred.join(movies_meta, on="movieId", how="left")

# 3. 사용자별 Top-K 추출
window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_proba").desc())

ranked = test_with_title.withColumn("rank", row_number().over(window_spec)) \
                        .filter(col("rank") <= K)

# 4. 추천 항목에 title 포함하여 구조화
ranked = ranked.withColumn(
    "recommendation",
    struct(
        col("movieId"),
        col("title"),
        col("y_pred_proba").alias("pred_rating")
    )
)

# 5. 사용자별 추천 리스트로 그룹화
recommendations = ranked.groupBy("userIndex") \
    .agg(collect_list("recommendation").alias("recommendations")) \
    .orderBy("userIndex")

# 6. 결과 확인
display(recommendations)

### MLFlow + Optuna

In [0]:
# 레이블 생성 (평점 4 이상은 1, 아니면 0)
# Optuna 목적 함수 정의
K = 10  # Precision@K 값

def objective(trial):
    param = {
        "maxDepth": trial.suggest_int("maxDepth", 3, 10),
        "eta": trial.suggest_float("eta", 0.01, 0.3),
        "numRound": trial.suggest_int("numRound", 50, 300),
        "objective": "binary:logistic",
        "evalMetric": "logloss",
        "earlyStoppingRounds": 20,
        "numWorkers": 4  # 작업자 수는 클러스터 환경에 맞게 조절
    }

    with mlflow.start_run(nested=True):
        mlflow.log_params(param)

        model = SparkXGBClassifier(
            featuresCol="features",  # VectorAssembler로 만든 feature 컬럼명
            labelCol="label",
            **param
        )

       # 모델 학습 (evalSets에 검증 데이터 전달)
        model = model.fit(train_df, evalSets=[valid_df])

        # 테스트 데이터 예측
        pred_test = model.transform(test_df)

        # positive 클래스 확률 추출 (probability 컬럼에서 [1])

        extract_prob = udf(lambda v: float(v[1]), DoubleType())
        pred_test = pred_test.withColumn("y_pred_prob", extract_prob(col("probability")))

        # Precision@K 계산
        window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_prob").desc())
        top_k = pred_test.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= K)

        # 평점 4 이상을 positive (1)로 변환
        top_k = top_k.withColumn("y_true", (col("rating") >= 4).cast("int"))

        precision_df = top_k.groupBy("userIndex").agg({"y_true": "avg"})
        avg_precision_at_k = precision_df.selectExpr("avg(`avg(y_true)`) as precision_at_k").collect()[0]["precision_at_k"]

        mlflow.log_metric("precision_at_k", avg_precision_at_k)
        mlflow.spark.log_model(model, "model")

        # Optuna는 최소화만 지원하므로 음수 반환
        return -avg_precision_at_k

In [0]:
# Optuna study 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# 최적 모델 학습 및 저장
best_model = XGBClassifier(**study.best_params)
best_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=20, verbose=False)

with mlflow.start_run(run_name="final_best_model"):
    mlflow.log_params(study.best_params)
    mlflow.sklearn.log_model(best_model, "final_model")

# 5. 한계

## 5-1. 안 본 영화 중에서 추천(rating 있다면 본 영화로 판단)

In [0]:
# 1. 사용자별 본 영화 집합 생성 (train_pd, validation_pd 기반 Spark DF로 변환 필요)
# 만약 train_df, valid_df가 Spark DF라면 바로 사용 가능
seen_movies_df = train_df.select("userIndex", "movieIndex") \
    .union(valid_df.select("userIndex", "movieIndex")) \
    .distinct() \
    .groupBy("userIndex") \
    .agg(collect_set("movieIndex").alias("seen_movies"))

# 2. test_pred: 예측 결과가 담긴 Spark DataFrame (컬럼: userIndex, movieIndex, y_pred_prob)
# test_pred와 seen_movies_df를 userIndex 기준으로 조인
test_pred_filtered = test_pred.join(seen_movies_df, on="userIndex", how="left_outer")

# 3. 이미 본 영화 제외 필터링 (seen_movies가 null일 수도 있으므로 null-safe)
test_pred_filtered = test_pred_filtered.filter(
    (col("seen_movies").isNull()) | (~array_contains(col("seen_movies"), col("movieIndex")))
)

# 4. 추천 상위 K개 추출
window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_prob").desc())

ranked = test_pred_filtered.withColumn("rank", row_number().over(window_spec)) \
                           .filter(col("rank") <= K)

# 5. 추천 결과 컬럼 생성 (movieId, pred_rating 구조체)
ranked = ranked.withColumn(
    "recommendation",
    struct(col("movieIndex").alias("movieId"), col("y_pred_prob").alias("pred_rating"))
)

# 6. 사용자별 추천 리스트 생성
recommendations = ranked.groupBy("userIndex") \
                        .agg(collect_list("recommendation").alias("recommendations")) \
                        .orderBy("userIndex")

display(recommendations)

## 5-2. 추가 User가 들어왔을때 어떤 영화를 추천할지

1. 신규유저에 영화정보를 바탕으로 기존유저와 매칭

2. 새로운 user_index 추천하지 않음(새로 학습해야함) - 현재 모델에서 user_index가 중요한 역할/ 처음 보는 User_index보고 엉뚱한 예측할 확률 높음

1. Cold Start 유저 처리
신규 유저:

A안: userIndex 제거한 일반 모델로 추천 제공 (장르, 평균 평점 기반)

B안: 기존 유저와 유사도 기반으로 매칭 → 그 유저의 추천을 가져옴 (collaborative filtering 유사)

2. 데이터 업데이트 및 재학습
신규 rating이 수집되면 데이터에 추가

일정 주기 (일간/주간 등)에 따라 모델 재학습

재학습된 모델은 최신화된 추천 반영 가능



1. 시청 영화 바탕으로 유사한 기존 유저 indexr를 찾고,
2. 그 indexr와 시청 영화 바탕으로 추천 영화 예측

In [0]:
# Step 1. 신규 유저가 본 영화 리스트 (Python set으로 준비)
new_user_seen_movies = [1, 10, 50, 300]
new_user_set = set(new_user_seen_movies)

# Step 2. 기존 유저별 시청 영화 집합 생성 (ratings는 Spark DataFrame)
user_movie_sets = ratings.groupBy("userId").agg(collect_set("movieId").alias("movie_set"))

# Step 3. Jaccard 유사도 계산 함수 정의 및 UDF 등록
def jaccard_similarity(movies):
    movies_set = set(movies) if movies is not None else set()
    intersection = len(new_user_set.intersection(movies_set))
    union = len(new_user_set.union(movies_set))
    return float(intersection) / union if union != 0 else 0.0

jaccard_udf = udf(jaccard_similarity, DoubleType())

# Step 4. 각 유저별 Jaccard 유사도 계산 후 내림차순 정렬, 최상위 1명 추출
similar_users = user_movie_sets.withColumn("jaccard_sim", jaccard_udf(col("movie_set"))) \
                               .orderBy(desc("jaccard_sim")) \
                               .limit(1)

# Step 5. 가장 유사한 유저의 userId 추출
top_user_id = similar_users.select("userId").first()["userId"]

# Step 6. user_indexer_model을 이용해 userIndex 변환 (userId -> userIndex)
top_user_index = user_indexer_model.transform(
    spark.createDataFrame([(top_user_id,)], ["userId"])
).select("userIndex").first()["userIndex"]

In [0]:
# Step 5. 전체 영화 목록에서 신규 유저가 보지 않은 영화만 필터링
unseen_movies_df = df_cleaned.filter(~col("movieId").isin(new_user_seen_movies))

# Step 6. 추천 후보 영화에 대해 pipeline_model 변환 (movieIndex, features 등 생성)
unseen_features_df = pipeline_model.transform(unseen_movies_df)

# Step 7. 유사한 기존 유저의 userIndex를 모든 행에 추가
unseen_features_df = unseen_features_df.withColumn("userIndex", lit(top_user_index))

# 중복 movieId 제거
unseen_features_unique_df = unseen_features_df.dropDuplicates(["movieId"])

# Step 8. features 컬럼을 numpy 배열로 변환 (pandas 변환 후 toArray 사용)
pdf_unseen = unseen_features_unique_df.select("features", "movieId", "title").toPandas()
X_unseen = np.vstack(pdf_unseen["features"].apply(lambda v: v.toArray()))

# Step 9. 모델 예측 (XGBClassifier 또는 XGBRegressor에 따라 predict 또는 predict_proba 사용)
# 여기서는 분류 문제인 경우로 가정해 predict_proba 사용, 클래스 1 확률 추출
y_pred_proba = model.predict_proba(X_unseen)[:, 1]

# Step 10. 예측 결과를 pandas DataFrame에 추가
pdf_unseen["prediction"] = y_pred_proba

# Step 11. 예측 상위 K개 추천 추출 (K=10)
top_k = pdf_unseen.sort_values(by="prediction", ascending=False).head(10)

# 결과 출력
print(top_k[["movieId", "title", "prediction"]])